<a href="https://colab.research.google.com/github/carolvieirav/operalabs/blob/main/freightech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import

In [62]:
!pip install selenium

In [63]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (95.0.4638.69-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [64]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

In [65]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

## Login

In [66]:
driver.get("https://freightech.ambev.com.br/#/login")

In [67]:
username = driver.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[1]/div/div/input")
password = driver.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-input[2]/div/div/input")

username.send_keys("anne.pereira@operalog.com.br")
password.send_keys("55026170")

driver.find_element_by_xpath("/html/body/app-root/ft-core-login/div/div/div/wac-button[1]/button").click()
print("Logged in Successfuly")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


Logged in Successfuly


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys


## Extracts the dashboard HTML source code

In [68]:
# set implicit wait time
driver.implicitly_wait(10) # seconds

driver.get("https://freightech.ambev.com.br/#/dashboard")
driver.find_element_by_xpath("/html/body/app-root/ft-home/div/main/div/ft-dashboard/div[1]/div[1]/div/wac-select").click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """


In [69]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
